In [1]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
import folium
import geopandas as gpd

# path to folder to CityGeoTools
folder = "/Users/lizzy/Documents/ИТМО/Прикладные методы анализа ГД/Online/CityGeoTools-master"
sys.path.append(folder)

from metrics.data import CityInformationModel as BaseModel

/Users/lizzy/Documents/ИТМО/Прикладные методы анализа ГД/Online/venv/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


### SpaceMatrix

The metric is designed to determine the type of urban area by a combination of indices: 
- Floor Space Index (FSI) - the ratio of the area of all floors of buildings and structures to the area of the block; 
- Ground Space Index (GSI) - the ratio of the area occupied by buildings and structures to the area of the block; 
- the average number of storeys of buildings in the quarter (L); 
- open space index (OSR) - the ratio of undeveloped space at ground level to the area of all floors of buildings and structures in the quarter; 
- Mixed use index (MXI) is defined as the ratio of the living area of all floors of buildings to the area of all floors of buildings and structures. 

To call the metric, first create your own City Information Model (the way how it is shown [here](https://github.com/iduprojects/CityGeoTools/blob/master/notebook_examples/create_city_information_model.ipynb)) and upload the [following files](https://github.com/iduprojects/CityGeoTools/tree/master/data_specification/visibility_analysis) into it:
- geojson file with urban buildings;
- geojson file with urban blocks.

In [3]:
city_model = BaseModel.CityInformationModel(city_name="Miass", city_crs=32641, cwd="../")
city_model.update_layer("Buildings", "/Users/lizzy/Documents/ИТМО/Прикладные методы анализа ГД/Online/building2.geojson")
city_model.update_layer("Blocks", "/Users/lizzy/Documents/ИТМО/Прикладные методы анализа ГД/Online/blocks.geojson")

Validation of Buildings layer...
Buildings layer loaded successfully!
Validation of Blocks layer...
Blocks layer loaded successfully!


In [4]:
city_model.methods.spacematrix.message

{'Buildings': 'Layer matches specification',
 'Blocks': 'Layer matches specification'}

In [5]:
from metrics.calculations import spacematrix
blocks_morphotypes = spacematrix.Spacematrix(city_model).get_morphotypes()
blocks_morphotypes = gpd.GeoDataFrame.from_features(blocks_morphotypes).set_crs(4326)

/Users/lizzy/Documents/ИТМО/Прикладные методы анализа ГД/Online/venv/lib/python3.9/site-packages/threadpoolctl.py:1019: RuntimeWarning: libc not found. The ctypes module in Python 3.9 is maybe too old for this OS.
  warnings.warn(


In [6]:
def style_fn(feature):
    colors = [
        'red', 'blue', 'green', 'orange', 'purple', 'pink', 'cadetblue', 'darkblue', 
        'lightgreen', 'darkred', 'yellow'
    ]
    return {'fillColor': colors[int(feature["properties"]["spacematrix_cluster"])]}

blocks_morphotypes_61 = blocks_morphotypes[blocks_morphotypes["administrative_unit_id"] == 1]
blocks_morphotypes_61 = blocks_morphotypes_61.dropna(subset=["spacematrix_cluster"])
map = folium.Map(location = [55.05661,60.11401], zoom_start = 13, tiles='CartoDB positron')
geo_j = folium.GeoJson(
    data=blocks_morphotypes_61, 
    style_function=style_fn,
    tooltip=folium.features.GeoJsonTooltip(["FSI", "GSI", "L", "MXI", "OSR", "spacematrix_cluster"])).add_to(map)
geo_j.add_to(map)
map